# 🚗 Smart Traffic System - Model Training

**Train XGBoost + LightGBM + Prophet on Google Colab (Free T4 GPU)**

---

## 📋 Steps:
1. Mount Google Drive
2. Load data (8,650 records)
3. Feature engineering (18+ features)
4. Train 3 models (~20 minutes)
5. Save models to Drive
6. Download to local machine

---

## 📦 Step 1: Install Libraries

In [ ]:
!pip install -q xgboost lightgbm prophet scikit-learn pandas numpy matplotlib seaborn plotly joblib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score, classification_report
import xgboost as xgb
import lightgbm as lgb
from prophet import Prophet
import joblib
import time

print("✅ Libraries imported")
print(f"XGBoost: {xgb.__version__}")
print(f"LightGBM: {lgb.__version__}")

## 📁 Step 2: Mount Google Drive & Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = '/content/drive/MyDrive/SmartTraffic/data/traffic_data_for_training.csv'
MODEL_OUTPUT_PATH = '/content/drive/MyDrive/SmartTraffic/models/'

print("✅ Drive mounted")

In [ ]:
print("📊 Loading data...")
df = pd.read_csv(DATA_PATH, parse_dates=['DateObservedFrom', 'DateObservedTo'])

print(f"✅ Loaded {len(df)} records")
print(f"Columns: {list(df.columns)}")
print(f"Shape: {df.shape}")
print(f"\nFirst 5 rows:")
df.head()

## 🔧 Step 3: Feature Engineering

In [ ]:
def create_features(df):
    df = df.copy()
    
    # Time features
    df['hour'] = df['DateObservedFrom'].dt.hour
    df['day_of_week'] = df['DateObservedFrom'].dt.dayofweek
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['is_rush_hour'] = ((df['hour'] >= 7) & (df['hour'] <= 9) | 
                           (df['hour'] >= 17) & (df['hour'] <= 19)).astype(int)
    
    # Sort
    df = df.sort_values(['RefRoadSegment', 'DateObservedFrom']).reset_index(drop=True)
    
    # Lag features
    df['speed_lag_1'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].shift(1)
    df['speed_lag_2'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].shift(2)
    df['speed_lag_3'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].shift(3)
    df['intensity_lag_1'] = df.groupby('RefRoadSegment')['Intensity'].shift(1)
    
    # Rolling statistics
    df['speed_rolling_mean_6'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].rolling(6, min_periods=1).mean().reset_index(0, drop=True)
    df['speed_rolling_mean_12'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].rolling(12, min_periods=1).mean().reset_index(0, drop=True)
    df['speed_rolling_std_6'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].rolling(6, min_periods=1).std().reset_index(0, drop=True)
    df['intensity_rolling_mean_6'] = df.groupby('RefRoadSegment')['Intensity'].rolling(6, min_periods=1).mean().reset_index(0, drop=True)
    
    # Difference features
    df['speed_diff'] = df.groupby('RefRoadSegment')['AverageVehicleSpeed'].diff()
    df['intensity_diff'] = df.groupby('RefRoadSegment')['Intensity'].diff()
    
    # Ratio
    df['speed_to_max_ratio'] = df['AverageVehicleSpeed'] / df['MaximumAllowedSpeed']
    
    # One-hot encoding
    segment_dummies = pd.get_dummies(df['RefRoadSegment'], prefix='segment')
    df = pd.concat([df, segment_dummies], axis=1)
    
    # Fill NaN
    df = df.fillna(method='bfill').fillna(method='ffill').fillna(0)
    
    return df

df_featured = create_features(df)
print(f"✅ Features created. Shape: {df_featured.shape}")
df_featured[['AverageVehicleSpeed', 'hour', 'is_rush_hour', 'speed_lag_1', 'speed_rolling_mean_6']].head(10)

## 🎯 Step 4: Prepare Train/Test Split

In [ ]:
feature_cols = [
    'Intensity', 'Occupancy', 'TotalLaneNumber', 'MaximumAllowedSpeed',
    'hour', 'day_of_week', 'is_weekend', 'is_rush_hour',
    'speed_lag_1', 'speed_lag_2', 'speed_lag_3', 'intensity_lag_1',
    'speed_rolling_mean_6', 'speed_rolling_mean_12', 'speed_rolling_std_6',
    'intensity_rolling_mean_6', 'speed_diff', 'intensity_diff', 'speed_to_max_ratio'
] + [col for col in df_featured.columns if col.startswith('segment_')]

X = df_featured[feature_cols]
y_speed = df_featured['AverageVehicleSpeed']
y_congestion = df_featured['Congested'].astype(int)

X_train, X_test, y_speed_train, y_speed_test, y_cong_train, y_cong_test = train_test_split(
    X, y_speed, y_congestion, test_size=0.2, random_state=42, shuffle=False
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Train: {len(X_train)} | Test: {len(X_test)}")
print(f"Features: {len(feature_cols)}")
print(f"\nCongestion - Train: Flowing={(y_cong_train==0).sum()}, Congested={(y_cong_train==1).sum()}")
print(f"Congestion - Test:  Flowing={(y_cong_test==0).sum()}, Congested={(y_cong_test==1).sum()}")

## 🚀 Step 5A: Train XGBoost (Congestion Classifier)

In [ ]:
print("="*80)
print("🔥 TRAINING XGBOOST")
print("="*80)

start = time.time()
scale_pos_weight = (y_cong_train == 0).sum() / (y_cong_train == 1).sum()

xgb_model = xgb.XGBClassifier(
    n_estimators=500, max_depth=8, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42, n_jobs=-1, tree_method='hist'
)

xgb_model.fit(X_train_scaled, y_cong_train, eval_set=[(X_test_scaled, y_cong_test)], verbose=50)

y_pred = xgb_model.predict(X_test_scaled)
train_time = time.time() - start

print(f"\n⏱️  Training: {train_time:.1f}s")
print(f"\n{classification_report(y_cong_test, y_pred, target_names=['Flowing', 'Congested'])}")

# Feature importance
feat_imp = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False).head(15)

print("\nTop 15 Features:")
print(feat_imp.to_string(index=False))

plt.figure(figsize=(10, 6))
plt.barh(feat_imp['feature'], feat_imp['importance'])
plt.xlabel('Importance')
plt.title('XGBoost Feature Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 🚀 Step 5B: Train LightGBM (Speed Regressor)

In [ ]:
print("="*80)
print("🔥 TRAINING LIGHTGBM")
print("="*80)

start = time.time()

lgb_model = lgb.LGBMRegressor(
    n_estimators=500, max_depth=8, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8,
    random_state=42, n_jobs=-1, verbose=-1
)

lgb_model.fit(X_train_scaled, y_speed_train, eval_set=[(X_test_scaled, y_speed_test)], callbacks=[lgb.log_evaluation(50)])

y_speed_pred = lgb_model.predict(X_test_scaled)
train_time = time.time() - start

mae = mean_absolute_error(y_speed_test, y_speed_pred)
rmse = np.sqrt(mean_squared_error(y_speed_test, y_speed_pred))
r2 = r2_score(y_speed_test, y_speed_pred)

print(f"\n⏱️  Training: {train_time:.1f}s")
print(f"\nMAE:  {mae:.2f} km/h")
print(f"RMSE: {rmse:.2f} km/h")
print(f"R²:   {r2:.4f}")

plt.figure(figsize=(10, 6))
plt.scatter(y_speed_test, y_speed_pred, alpha=0.3)
plt.plot([y_speed_test.min(), y_speed_test.max()], [y_speed_test.min(), y_speed_test.max()], 'r--', lw=2)
plt.xlabel('Actual Speed (km/h)')
plt.ylabel('Predicted Speed (km/h)')
plt.title(f'LightGBM: MAE={mae:.2f}, R²={r2:.3f}')
plt.tight_layout()
plt.show()

## 🚀 Step 5C: Train Prophet (Per Segment)

In [ ]:
print("="*80)
print("🔥 TRAINING PROPHET")
print("="*80)

prophet_models = {}
segments = df_featured['RefRoadSegment'].unique()
start = time.time()

for segment in segments:
    print(f"\n📊 {segment}...")
    df_seg = df_featured[df_featured['RefRoadSegment'] == segment].copy()
    
    df_prophet = pd.DataFrame({
        'ds': df_seg['DateObservedFrom'],
        'y': df_seg['AverageVehicleSpeed'],
        'intensity': df_seg['Intensity'],
        'is_weekend': df_seg['is_weekend'],
        'is_rush_hour': df_seg['is_rush_hour']
    })
    
    model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=False)
    model.add_regressor('intensity')
    model.add_regressor('is_weekend')
    model.add_regressor('is_rush_hour')
    model.fit(df_prophet)
    
    prophet_models[segment] = model
    print(f"  ✅ Done")

train_time = time.time() - start
print(f"\n⏱️  Total: {train_time:.1f}s")
print(f"✅ {len(prophet_models)} models trained")

## 💾 Step 6: Save Models to Drive

In [ ]:
import os
os.makedirs(MODEL_OUTPUT_PATH, exist_ok=True)

print("="*80)
print("💾 SAVING MODELS")
print("="*80)

xgb_path = os.path.join(MODEL_OUTPUT_PATH, 'xgboost_congestion.pkl')
joblib.dump(xgb_model, xgb_path)
print(f"✅ XGBoost: {os.path.getsize(xgb_path)/1024:.1f} KB")

lgb_path = os.path.join(MODEL_OUTPUT_PATH, 'lightgbm_speed.pkl')
joblib.dump(lgb_model, lgb_path)
print(f"✅ LightGBM: {os.path.getsize(lgb_path)/1024:.1f} KB")

prophet_path = os.path.join(MODEL_OUTPUT_PATH, 'prophet_models.pkl')
joblib.dump(prophet_models, prophet_path)
print(f"✅ Prophet: {os.path.getsize(prophet_path)/1024:.1f} KB")

scaler_path = os.path.join(MODEL_OUTPUT_PATH, 'scaler.pkl')
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler: {os.path.getsize(scaler_path)/1024:.1f} KB")

feat_path = os.path.join(MODEL_OUTPUT_PATH, 'feature_columns.pkl')
joblib.dump(feature_cols, feat_path)
print(f"✅ Features: {os.path.getsize(feat_path)/1024:.1f} KB")

print("\n🎉 ALL SAVED TO GOOGLE DRIVE!")
print(f"Location: {MODEL_OUTPUT_PATH}")
print("\nNext: Download to local machine")

## 📊 Step 7: Summary

In [ ]:
print("="*80)
print("📊 TRAINING SUMMARY")
print("="*80)

summary = pd.DataFrame([
    {'Model': 'XGBoost', 'Task': 'Congestion', 'Metric': f"{f1_score(y_cong_test, y_pred):.3f} F1", 'File': 'xgboost_congestion.pkl'},
    {'Model': 'LightGBM', 'Task': 'Speed', 'Metric': f"{mae:.2f} km/h MAE", 'File': 'lightgbm_speed.pkl'},
    {'Model': f'Prophet x{len(prophet_models)}', 'Task': 'Trend', 'Metric': 'Per-segment', 'File': 'prophet_models.pkl'}
])

print(summary.to_string(index=False))
print("\n✅ Training complete!")
print("📁 Models saved to Google Drive")
print("🚀 Ready for deployment")